In [7]:
import pandas as pd
import numpy as np
from pulp import *
import warnings
warnings.filterwarnings('ignore')

# Load your data

xlsx_path=r"C:\Users\Pratham Borkar\Desktop\data a\cleaned_castle_of_insights.csv"
df=pd.read_csv(xlsx_path)


# Clean column names
df.columns = df.columns.str.strip()

# Calculate output weights (A=3, B=2, C=1)
# Based on: A does 50% more than B, B does 2x more than C
# If C=1, then B=2, then A=3 (2*1.5=3)
df['weight'] = df['Work_Rating'].map({'A': 3, 'B': 2, 'C': 1})
df['output_est'] = df['weight'] * 3  # Normalized output units
df['output_per_cost'] = df['output_est'] / (df['Salary'] / 10000)  # Your formula

# Calculate tenure flags
df['senior'] = (df['Tenure'] > 15).astype(int)
df['experienced'] = (df['Tenure'] > 5).astype(int)

print("="*80)
print("BASELINE METRICS (Before Optimization)")
print("="*80)

# Baseline calculations
total_employees = len(df)
total_salary = df['Salary'].sum()
total_output = df['output_est'].sum()
output_cost_ratio = total_output / (total_salary / 1000000)

print(f"\nTotal Employees: {total_employees}")
print(f"Total Salary: ₹{total_salary:,.0f}")
print(f"Total Output: {total_output:,.0f} units")
print(f"Output-to-Cost Ratio: {output_cost_ratio:.2f}")

print(f"\nCompany Distribution:")
print(df['Company_Origin'].value_counts())

print(f"\nWork Rating Distribution:")
rating_counts = df['Work_Rating'].value_counts()
print(rating_counts)

print(f"\nTenure Distribution:")
print(f"Senior (>15 years): {df['senior'].sum()}")
print(f"Experienced (>5 years): {df['experienced'].sum()}")

# Calculate Technova and Dataverse individual outputs
technova_output = df[df['Company_Origin'] == 'Technova']['output_est'].sum()
dataverse_output = df[df['Company_Origin'] == 'Dataverse']['output_est'].sum()
print(f"\nTechnova Output: {technova_output:,.0f}")
print(f"Dataverse Output: {dataverse_output:,.0f}")
max_individual_output = max(technova_output, dataverse_output)

print("\n" + "="*80)
print("OPTIMIZATION CONSTRAINTS")
print("="*80)

# Define constraints
MAX_EMPLOYEES = 800
MAX_SALARY = total_salary * 0.90  # 10% reduction
MIN_OUTPUT = max_individual_output + 1  # Must exceed highest individual company

MIN_A_RETAIN = int(np.ceil(rating_counts.get('A', 0) * 0.60))
MIN_B_RETAIN = int(np.ceil(rating_counts.get('B', 0) * 0.30))
MIN_SENIOR = int(np.ceil(df['senior'].sum() * 0.05))
MIN_EXPERIENCED = int(np.ceil(df['experienced'].sum() * 0.15))

print(f"\n1. Employees: 750-800 (target MAX 80% = {MAX_EMPLOYEES})")
print(f"2. Max Salary: ₹{MAX_SALARY:,.0f} (10% reduction)")
print(f"3. Min Output: {MIN_OUTPUT:,.0f} (must exceed {max_individual_output:,.0f})")
print(f"4. Min A-rated: {MIN_A_RETAIN} (60% of {rating_counts.get('A', 0)})")
print(f"5. Min B-rated: {MIN_B_RETAIN} (30% of {rating_counts.get('B', 0)})")
print(f"6. Min Senior: {MIN_SENIOR} (5% of {df['senior'].sum()})")
print(f"7. Min Experienced: {MIN_EXPERIENCED} (15% of {df['experienced'].sum()})")
print(f"8. Each department: 50% from each company (min)")

print("\n" + "="*80)
print("RUNNING OPTIMIZATION MODEL")
print("="*80)

# Create optimization problem
prob = LpProblem("Workforce_Optimization", LpMaximize)

# Decision variables: 1 if employee is retained, 0 otherwise
retain = {i: LpVariable(f"retain_{i}", cat='Binary') for i in df.index}

# Objective: Maximize output while staying within salary budget and using full headcount
# Priority: 1) Maximize output, 2) Use full 800 employees, 3) Minimize cost
prob += (
    lpSum([retain[i] * df.loc[i, 'output_est'] for i in df.index]) * 1000 +
    lpSum([retain[i] for i in df.index]) * 100 -
    lpSum([retain[i] * df.loc[i, 'Salary'] for i in df.index]) * 0.01
)

# Constraint 1: Retain between 750-800 employees (aim for maximum 80%)
# We want to use the full 80% allowance, not minimize headcount
prob += lpSum([retain[i] for i in df.index]) >= 750  # Minimum 75%
prob += lpSum([retain[i] for i in df.index]) <= MAX_EMPLOYEES  # Maximum 80%

# Constraint 2: Max 90% of original salary (10% reduction)
prob += lpSum([retain[i] * df.loc[i, 'Salary'] for i in df.index]) <= MAX_SALARY

# Constraint 3: Output must exceed best individual company
prob += lpSum([retain[i] * df.loc[i, 'output_est'] for i in df.index]) >= MIN_OUTPUT

# Constraint 4: Min A-rated employees (60%)
prob += lpSum([retain[i] for i in df[df['Work_Rating'] == 'A'].index]) >= MIN_A_RETAIN

# Constraint 5: Min B-rated employees (30%)
prob += lpSum([retain[i] for i in df[df['Work_Rating'] == 'B'].index]) >= MIN_B_RETAIN

# Constraint 6: Min senior employees (5%)
prob += lpSum([retain[i] for i in df[df['senior'] == 1].index]) >= MIN_SENIOR

# Constraint 7: Min experienced employees (15%)
prob += lpSum([retain[i] for i in df[df['experienced'] == 1].index]) >= MIN_EXPERIENCED

# Constraint 8: Department-level company balance (40-60 minimum, aiming for 50-50)
# Each company should have AT LEAST 40% representation in each department
departments = df['Department'].unique()
for dept in departments:
    dept_df = df[df['Department'] == dept]
    dept_technova = dept_df[dept_df['Company_Origin'] == 'Technova']
    dept_dataverse = dept_df[dept_df['Company_Origin'] == 'Dataverse']
    
    # Total retained in department
    total_dept = lpSum([retain[i] for i in dept_df.index])
    
    # Each company gets at least 40% (which means max 60% for the other)
    # This ensures fair representation while being mathematically feasible
    if len(dept_technova) > 0:
        prob += lpSum([retain[i] for i in dept_technova.index]) >= 0.4 * total_dept
    if len(dept_dataverse) > 0:
        prob += lpSum([retain[i] for i in dept_dataverse.index]) >= 0.4 * total_dept

print("\nSolving optimization problem...")
prob.solve(PULP_CBC_CMD(msg=0))

print(f"Status: {LpStatus[prob.status]}")

# Extract results
df['retained'] = df.index.map(lambda i: retain[i].varValue)

retained_df = df[df['retained'] == 1].copy()
terminated_df = df[df['retained'] == 0].copy()

print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)

new_employees = len(retained_df)
new_salary = retained_df['Salary'].sum()
new_output = retained_df['output_est'].sum()
new_ratio = new_output / (new_salary / 1000000)

salary_saved = total_salary - new_salary
salary_reduction_pct = (salary_saved / total_salary) * 100
output_increase = new_output - max_individual_output
output_increase_pct = (output_increase / max_individual_output) * 100

print(f"\n✓ Employees Retained: {new_employees} / {MAX_EMPLOYEES} (target 750-800)")
print(f"✓ Employees Terminated: {len(terminated_df)}")
print(f"✓ Retention Rate: {(new_employees/total_employees)*100:.1f}%")
print(f"✓ Total Salary: ₹{new_salary:,.0f} (saved ₹{salary_saved:,.0f}, {salary_reduction_pct:.1f}%)")
print(f"✓ Total Output: {new_output:,.0f} units (increased by {output_increase:.0f}, {output_increase_pct:.1f}%)")
print(f"✓ Output-to-Cost Ratio: {new_ratio:.2f} (was {output_cost_ratio:.2f}, improved by {((new_ratio/output_cost_ratio - 1)*100):.1f}%)")

print(f"\n✓ A-rated retained: {len(retained_df[retained_df['Work_Rating']=='A'])} / {MIN_A_RETAIN} required")
print(f"✓ B-rated retained: {len(retained_df[retained_df['Work_Rating']=='B'])} / {MIN_B_RETAIN} required")
print(f"✓ C-rated retained: {len(retained_df[retained_df['Work_Rating']=='C'])}")

print(f"\n✓ Senior retained: {retained_df['senior'].sum()} / {MIN_SENIOR} required")
print(f"✓ Experienced retained: {retained_df['experienced'].sum()} / {MIN_EXPERIENCED} required")

print(f"\n✓ Technova retained: {len(retained_df[retained_df['Company_Origin']=='Technova'])}")
print(f"✓ Dataverse retained: {len(retained_df[retained_df['Company_Origin']=='Dataverse'])}")

# Department-wise breakdown
print("\n" + "="*80)
print("DEPARTMENT-WISE BREAKDOWN")
print("="*80)

dept_summary = retained_df.groupby('Department').agg({
    'Employee_ID': 'count',
    'Salary': 'sum',
    'output_est': 'sum'
}).rename(columns={'Employee_ID': 'Count'})

dept_company = retained_df.groupby(['Department', 'Company_Origin']).size().unstack(fill_value=0)
dept_summary = dept_summary.join(dept_company)

print("\n", dept_summary)

# Verify 50-50 constraint
print("\n" + "="*80)
print("VERIFYING COMPANY BALANCE PER DEPARTMENT (Target: 40-60% minimum)")
print("="*80)

for dept in departments:
    dept_retained = retained_df[retained_df['Department'] == dept]
    total = len(dept_retained)
    technova_count = len(dept_retained[dept_retained['Company_Origin'] == 'Technova'])
    dataverse_count = len(dept_retained[dept_retained['Company_Origin'] == 'Dataverse'])
    
    if total > 0:
        tech_pct = (technova_count / total) * 100
        data_pct = (dataverse_count / total) * 100
        
        # Check if both are at least 40% (fair representation)
        status = "✓" if tech_pct >= 40 and data_pct >= 40 else "✗"
        balance_status = "BALANCED" if 45 <= tech_pct <= 55 else "ACCEPTABLE" if 40 <= tech_pct <= 60 else "IMBALANCED"
        
        print(f"{status} {dept}: Total={total} | Tech={technova_count} ({tech_pct:.1f}%), Data={dataverse_count} ({data_pct:.1f}%) [{balance_status}]")

# Export results
print("\n" + "="*80)
print("EXPORTING RESULTS")
print("="*80)

# 1. Excel with Employee IDs only (no header)
retained_ids = retained_df[['Employee_ID']].copy()
retained_ids.to_csv('retained_employees.csv', index=False, header=False)
print("\n✓ Saved: retained_employees.csv (Employee IDs only, no header)")

# 2. Detailed analysis file for your reference
output_file = 'optimization_analysis.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    retained_df.to_excel(writer, sheet_name='Retained_Employees', index=False)
    terminated_df.to_excel(writer, sheet_name='Terminated_Employees', index=False)
    dept_summary.to_excel(writer, sheet_name='Department_Summary')
    
    # Summary sheet
    summary_data = {
        'Metric': [
            'Total Employees (Before)', 'Total Employees (After)', 'Reduction',
            'Total Salary (Before)', 'Total Salary (After)', 'Cost Savings (%)',
            'Total Output (Before)', 'Total Output (After)', 'Output Increase (%)',
            'Output-to-Cost Ratio (Before)', 'Output-to-Cost Ratio (After)',
            'A-rated Retained', 'B-rated Retained', 'C-rated Retained',
            'Senior Retained', 'Experienced Retained',
            'Technova Retained', 'Dataverse Retained'
        ],
        'Value': [
            total_employees, new_employees, total_employees - new_employees,
            f"₹{total_salary:,.0f}", f"₹{new_salary:,.0f}", f"{salary_reduction_pct:.1f}%",
            f"{total_output:,.0f}", f"{new_output:,.0f}", f"{output_increase_pct:.1f}%",
            f"{output_cost_ratio:.2f}", f"{new_ratio:.2f}",
            len(retained_df[retained_df['Work_Rating']=='A']),
            len(retained_df[retained_df['Work_Rating']=='B']),
            len(retained_df[retained_df['Work_Rating']=='C']),
            retained_df['senior'].sum(),
            retained_df['experienced'].sum(),
            len(retained_df[retained_df['Company_Origin']=='Technova']),
            len(retained_df[retained_df['Company_Origin']=='Dataverse'])
        ]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

print(f"✓ Saved: {output_file} (detailed analysis for Power BI)")

print("\n" + "="*80)
print("OPTIMIZATION COMPLETE!")
print("="*80)
print("\nFiles generated:")
print("1. retained_employees.csv - Submit this (Employee IDs only)")
print("2. optimization_analysis.xlsx - Use for Power BI dashboard")
print("\nYou can now create your Power BI dashboard using the Excel file!")

BASELINE METRICS (Before Optimization)

Total Employees: 1000
Total Salary: ₹40,946,000
Total Output: 5,757 units
Output-to-Cost Ratio: 140.60

Company Distribution:
Technova     535
Dataverse    465
Name: Company_Origin, dtype: int64

Work Rating Distribution:
B    601
C    240
A    159
Name: Work_Rating, dtype: int64

Tenure Distribution:
Senior (>15 years): 399
Experienced (>5 years): 801

Technova Output: 3,084
Dataverse Output: 2,673

OPTIMIZATION CONSTRAINTS

1. Employees: 750-800 (target MAX 80% = 800)
2. Max Salary: ₹36,851,400 (10% reduction)
3. Min Output: 3,085 (must exceed 3,084)
4. Min A-rated: 96 (60% of 159)
5. Min B-rated: 181 (30% of 601)
6. Min Senior: 20 (5% of 399)
7. Min Experienced: 121 (15% of 801)
8. Each department: 50% from each company (min)

RUNNING OPTIMIZATION MODEL

Solving optimization problem...
Status: Optimal

OPTIMIZATION RESULTS

✓ Employees Retained: 800 / 800 (target 750-800)
✓ Employees Terminated: 200
✓ Retention Rate: 80.0%
✓ Total Salary: ₹36,